# Lab: Train a Tiny Neural Language Model 

## Goal
Train a small GRU-based language model that learns meaningful next-token predictions,
while remaining CPU-friendly.

### You will:
1. Load WikiText-2
2. Build fixed-length training blocks
3. Implement a Tiny GRU LM
4. Train for a few epochs
5. Measure perplexity
6. Inspect top-k next-token predictions


In [ ]:
!pip -q install datasets transformers
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## 1️⃣ Load WikiText-2
We use a small slice to stay fast.

**TODO:**
- Load first 5000 examples
- Remove empty lines
- Create train/val split (90/10)
- Load GPT-2 tokenizer

In [ ]:
# TODO
# ds = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train[:5000]')
# texts = ...
# split = int(0.9 * len(texts))
# train_texts = ...
# val_texts = ...

# tokenizer = AutoTokenizer.from_pretrained('gpt2')
# tokenizer.pad_token = tokenizer.eos_token
# len(train_texts), len(val_texts)

## 2️⃣ Build Language Modeling Blocks
Concatenate tokens and split into blocks of length 64.

For each block:
- Inputs: block[:, :-1]
- Labels: block[:, 1:]

**TODO:** Complete function.

In [ ]:
def build_lm_blocks(texts, tokenizer, block_size=64):
    all_ids = []
    
    # TODO: tokenize and concatenate
    
    # TODO: drop remainder
    # TODO: reshape into blocks
    
    return blocks

# TODO: create train_blocks and val_blocks

## 3️⃣ Create DataLoaders
Batch size 32 (CPU-friendly).

In [ ]:
# TODO
# train_loader = DataLoader(train_blocks, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_blocks, batch_size=32)
# next(iter(train_loader)).shape

## 4️⃣ Implement Tiny GRU LM
Embedding(128) → GRU(256) → Linear

In [ ]:
class TinyGRULM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        # TODO

    def forward(self, input_ids):
        # TODO
        return logits

# model = TinyGRULM(len(tokenizer)).to(device)

## 5️⃣ Train the Model
- 5 epochs
- CrossEntropyLoss
- Gradient clipping (1.0)

Perplexity = exp(loss)

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for block in loader:
        block = block.to(device)
        inputs = block[:, :-1]
        labels = block[:, 1:]

        # TODO: forward
        # TODO: loss
        # TODO: backward + clip + step

        total_loss += loss.item()
    return total_loss / len(loader)

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    for block in loader:
        block = block.to(device)
        inputs = block[:, :-1]
        labels = block[:, 1:]
        # TODO: forward + loss
        total_loss += loss.item()
    return total_loss / len(loader)

# TODO: train for 5 epochs and print train/val perplexity

## 6️⃣ Inspect Top‑5 Next Tokens
Now use your trained model to inspect its prediction for the next token after a prompt.

Inside topk_next_tokens(...), you should:

- Tokenize the prompt into input_ids
- Run the model once to obtain logits
- Select the logits at the last time step (this corresponds to the next-token prediction)
- Use torch.topk to find the top-5 token IDs
- Decode those token IDs back to text
- Return a list of (token_string, score) pairs

You do not need to apply softmax or generate multiple tokens.
We only inspect the model’s single-step next-token distribution.

In [ ]:
@torch.no_grad()
def topk_next_tokens(model, tokenizer, prompt, k=5):
    model.eval()
    # TODO: tokenize prompt
    # TODO: forward
    # TODO: topk from last timestep
    # TODO: decode
    return results

# prompt = "Sheffield is the"
# print(topk_next_tokens(model, tokenizer, prompt))